In [1]:
import clustertools as cts
from clustertools.analysis.functions import meq_func
import numpy as np
import matplotlib.pyplot as plt

/Users/webbjere/Codes/clustertools/clustertools/analysis/profiles.py:37: FutureWarning: all profiles are setup such that the returned radial bins and profile values are in linear space and not normalized by the effective radius. Previously select profiles had unique returns.
  warnings.warn('all profiles are setup such that the returned radial bins and profile values are in linear space and not normalized by the effective radius. Previously select profiles had unique returns.',FutureWarning)



# Functions

Load a snapshot of a cluster in file 00000.dat, which has position units of pc and velocity units of km/s in clustercentric coordinates. Stellar masses are in solar units and were generated using a Salpeter IMF.

In [2]:
cluster=cts.load_cluster('snapshot',filename='00000.dat',units='pckms',origin='cluster',ofilename='orbit.dat',ounits='kpckms')

Once initialized, any function within ``clustertools`` can be called internally via ``cluster.function_name()`` or externally via ``output=function_name(cluster)``. An internal call sets variables within ``cluster`` while an external call will return the calculated values but change nothing within ``cluster``.

In the event that the snapshot is not centered, the centre of the cluster can be found via:

In [3]:
cluster.find_centre()

(-0.14946119556654258,
 -0.22087743738380078,
 -0.038780116570714403,
 0.014544487149392757,
 -0.097072929729564258,
 -0.028231116712277449)

which not only returns the position and velocity of the centre of mass, but also sets ``cluster.xc``, ``cluster.yc``, ``cluster.zc``, ``cluster.vxc``, ``cluster.vyc``, ``cluster.vzc`` Alternatively, if you don't want to set any ``StarCluster`` variables just call the function externally:

In [4]:
xc,yc,zc,vxc,vyc,vzc=cts.find_centre(cluster)
print(xc,yc,zc)

-0.149461195567 -0.220877437384 -0.0387801165707


It is important to note that, by default, the centre is calculated to be the centre of density. Similar to NBODY6 and phigrape, once the centre of density is found for the entire cluster population, a centralized subset of stars within an ever decreasing radius are used to find the true centre of density. The parameters ``rmin`` and ``nmax`` set the minimum radius that can encompass the subset of stars and the maximum number of stars within the subset. 

Setting ``density=False`` will instead find a central subset of stars to find the cluster's of centre by removing stars beyond ``nsigma`` standard deviations of the previously calculated centre. For systems with a large number of escaped stars, which make finding the cluster's centre difficult, it may also help to tell the function where to start. For example, it is possible to tell the cluster to start looking for the centre 1 pc away from the origin along the x-axis and remove stars beyond two standard deviations via:

In [5]:
cluster.find_centre(xstart=1.,density=False, nsigma=1)

(0.47255599444213536,
 -0.046784905321136626,
 0.03368417753435262,
 0.0073060547641532469,
 -0.05167863301943116,
 -0.0025985741474006487)

Other functions that can be called include:

In [6]:
print('Half-Mass Relaxation Time: ',cluster.half_mass_relaxation_time())
print('Core Relaxation Time: ',cluster.core_relaxation_time())
print('Lagrange Radii: ',cluster.rlagrange())
print('Virial Radius: ',cluster.virial_radius())

Half-Mass Relaxation Time:  50.5231000207
Core Relaxation Time:  19.6957365444
Lagrange Radii:  [0.73326598977617097, 1.0765583789354674, 1.3646287125197105, 1.7124821982817706, 1.9617869129434311, 2.3496933983866723, 3.1257216188480137, 3.9773141578345004, 5.0211933936198365, 8.3242591110873079]
Virial Radius:  2.51937714706


All functions allow for projected values to be used instead:

In [7]:
print('Half-Mass Relaxation Time: ',cluster.half_mass_relaxation_time(projected=True))
print('Core Relaxation Time: ',cluster.core_relaxation_time(projected=True))
print('Lagrange Radii: ',cluster.rlagrange(projected=True))
print('Virial Radius: ',cluster.virial_radius(projected=True))

Half-Mass Relaxation Time:  33.8815120123
Core Relaxation Time:  20.6789810497
Lagrange Radii:  [0.49097752209716733, 0.7013820946414554, 0.95595297621692432, 1.2127055798991366, 1.5030278513713957, 1.8280543898615964, 2.3000812878962309, 3.068226381254116, 4.1294671402214904, 8.1792344411375115]
Virial Radius:  1.57999162536


The core radius of the cluster can be calculated by finding where the density drops to 1/3 the central value. The parameter ``mfrac`` sets the mass fraction of the cluster that defintes the central region (default 0.1). When ``projected=True`` the core radius is where the density drops to 1/2 the central value. ``rcore`` also has a plotting feature

In [8]:
cluster.rcore(plot=True)
print(cluster.rc)

DEBUG:  [ 0.1853306   0.79224147  0.36840963  0.29903793  0.18726088  0.15207877
  0.11834952  0.10860732  0.12008112  0.14971781  0.31200067  0.86651931
  0.14591688  0.17578746  0.21194402  0.42654811  0.32461966  0.17611346
  0.15401563  0.19694323  0.34209816  0.11531121  0.11477758  0.10771665
  0.16204727  0.11044941  0.125415    0.11122382  0.14123954  0.92706062
  0.12697832  0.12973868  0.13635222  0.13674444  0.10059388  0.11701243
  0.10376417  0.13753034  0.10000612  0.11098097  0.11090943  0.47055522
  0.16747204  0.13212422  0.62693001  0.11122248  1.1697209   0.20025021
  0.12001291  0.75591621  0.14457091  0.13599219  0.16297219  2.028748
  0.26918135  0.16973077  0.1632298   0.12108362  0.28057275  0.23389789
  0.4568857   0.79372582  0.10760674  0.14950178  0.46545551  0.18618395
  0.16507853  0.15885858  0.12114283  0.15093151  0.22513159  0.66446436
  7.47728527  0.14662078  0.13019611  3.04660051  0.11649679  0.11348659
  0.22379911  0.10973708  5.02300025  0.22607

IndexError: invalid index to scalar variable.

It is also possible to easily measure the mass function and its slope within a given mass range:

In [ ]:
m_mean, m_hist, dm, alpha, ealpha, yalpha, eyalpha=cts.mass_function(cluster,mmin=0.1,mmax=0.8,plot=True)

Calling the mass function function internally simply sets the value of alpha and saves the mass range over which alpha was measured with cluster.mmin and cluster.mmax 

In [ ]:
cluster.mass_function(mmin=0.1,mmax=0.8)

Looking at how velocity dispersion changes as a function of stellar mass is also a proxy for a cluster's dynamical state, with the power-law slope eta evolving towards (but never reaching) -0.5. An eta of -0.5 corresponds to a state of complete energy equipartition. The external call to ``eta_function`` (shown below) and the internal call behave similarly to ``mass_function`` above.

In [ ]:
m_mean, sigvm, eta, eeta, yeta, eyeta=cts.eta_function(cluster,plot=True)

Alternatively, as per Bianchini et al. 2016, the state of energy equipartition can be probed by the meq parameter

In [ ]:
m_mean, sigvm, meq, emeq, ymeq, eymeq=cts.meq_function(cluster,plot=True)

Similarly, as per Bianchini et al. 2018, MNRAS, 475, 96, if you can measure meq you can also measure the kinematic concentration ck, which is the ratio of meq within the half-mass radius to meq at the half mass radius:

In [ ]:
cluster.ckin()

If the cluster's galactocentric position and velocity are known, then its tidal radius and limiting radius can also be determined. Note when calculating the limiting radius, it is possible to select ``plot=True`` to see how the cluster's density profile reaches the background density. 

Note that you need to specify a ``galpy`` potential in order to calclated these values. For a Milky Way-like external tidal field, MWPotential2014 from Bovy J., 2015, ApJS, 216, 29 is useful. Different potentials can be used by assigning a ``galpy`` potential to the ``pot`` variable.


In [ ]:
from galpy.potential import MWPotential2014

print('Tidal Radius: ',cluster.rtidal(pot=MWPotential2014))
print('Limiting Radius: ',cluster.rlimiting(pot=MWPotential2014,plot=True))

It is also worth noting that it is possible to calculate the tidal radius iteratively. For example, lets assume a snapshot also contains a large number of stars that are likely not members of the cluster. By setting ``rtiterate>0``, ``clustertools`` will first calculate the tidal radius using the mass of all stars in the snapshot (rt_old). However it will then calculate the tidal radius using only the mass of stars within the initial calculation (rt_new). This process is repeated ``rtiterate`` times or until the ratio of rt_new/rt_old is not less than ``rtconverge``, which by default is 0.9.

In [ ]:
print('Tidal Radius: ',cluster.rtidal(pot=MWPotential2014,rtiterate=10))

Since not all snapshot will contain the energy of each star, ``clustertools`` can calculate these values as well by simply calling:

In [ ]:
ek, pot=cts.energies(cluster)

If energies were instead called internally, such that ``cluster.ek``,``cluster.pot``,``cluster.etot`` are set, a subset of stars could be defined that are gravitationally bound (E<0):

In [ ]:
cluster.energies()
bound_cluster=cts.sub_cluster(cluster,emax=0.)

Calling ``energies`` internally also set the virial parameter Qvir:

In [ ]:
print('Qvir = ',cluster.qvir)

Finally, in case one needs to know the distance to each stars nearest neighbour:

In [ ]:
distance_to_neighbour=cts.closest_star(cluster)
print(np.amin(distance_to_neighbour))